In [1]:
import pandas as pd
from unidecode import unidecode

In [3]:
df_model = pd.read_csv('data/process/TSE.csv', encoding='latin1')
df_model.head()

,nome,sexo,primeiro_nome
0,CLEMILDO BARBOSA LIMA,MASCULINO,clemildo
1,MARIA AUXILIADORA DIAS,FEMININO,maria
2,FRANCISCO CLODOALDO DE SOUZA RODRIGUES,MASCULINO,francisco
3,MARIA DAS GRAÇAS ALVES DA SILVA,FEMININO,maria
4,RAILA INACIO CORREIA,FEMININO,raila


In [4]:
df_model['sexo_cat'] = df_model['sexo'].apply(lambda x: -1 if x == 'MASCULINO' else 1)

In [5]:
df_model = df_model[['primeiro_nome', 'sexo']]
df_model.head()

,primeiro_nome,sexo
0,clemildo,MASCULINO
1,maria,FEMININO
2,francisco,MASCULINO
3,maria,FEMININO
4,raila,FEMININO


NORMALISE NAMES

In [6]:
df_model['primeiro_nome'] = df_model['primeiro_nome'].apply(lambda x: unidecode(x).lower())

Show most commom names

In [7]:
df_model.groupby('primeiro_nome').count().sort_values(by='sexo', ascending=False).head()

,sexo
primeiro_nome,
jose,28668
maria,21227
antonio,11972
joao,9958
francisco,8097


Get probability of the sex

In [8]:
def sex_prob(x):
    
    masc = (x['sexo'] == 'MASCULINO').sum()
    fem = (x['sexo'] == 'FEMININO').sum()

    return (masc - fem) / float(len(x))

In [9]:
df = df_model.groupby('primeiro_nome').apply(sex_prob)

In [10]:
df.head()

primeiro_nome
.jose     1.0
aarao     1.0
aaron     1.0
abadi     1.0
abadia   -1.0
dtype: float64

In [11]:
# Write file
df.to_csv('data/process/nome_sexo.csv', encoding='latin1')

Federal Data Cleaning

In [14]:
df_data = pd.read_csv('data/process/primeiro_nome.csv', index_col='Unnamed: 0')
print 'Tamanho da base', len(df_data)
df_data.head()

Tamanho da base 590670


,NOME,ID_SERVIDOR_PORTAL,REMUNERACAO_BASICA_BRUTA,PRIMEIRO_NOME
0,PAULO MARTINS SOARES,1000000,"6779,03",PAULO
1,ANALIA BATISTA,1000001,"9151,22",ANALIA
2,ARY DE NOROES FILHO,1000002,"6012,61",ARY
3,MOISES ALBERTO CALLE AGUIRRE,1000004,"11554,56",MOISES
4,LAURA JANAINA DIAS AMATO,1000005,"10769,82",LAURA


Show most commom names

In [15]:
df_data.groupby('PRIMEIRO_NOME').count().sort_values(by='ID_SERVIDOR_PORTAL', ascending=False).head()

,NOME,ID_SERVIDOR_PORTAL,REMUNERACAO_BASICA_BRUTA
PRIMEIRO_NOME,,,
MARIA,25956,25956,25956
JOSE,20172,20172,20172
ANA,9655,9655,9655
ANTONIO,8915,8915,8915
CARLOS,8350,8350,8350


Normalise names

In [16]:
df_data['PRIMEIRO_NOME'] = df_data['PRIMEIRO_NOME'].apply(lambda x: unidecode(x).lower())

/Users/Admin/anaconda/lib/python2.7/site-packages/unidecode/__init__.py:46: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  _warn_if_not_unicode(string)


Match names with sex

In [17]:
def get_sex(x, **kwargs):
    
    if x in df.keys():
        return df[x]
    else:
        return 0

In [18]:
df_data['sexo'] = df_data['PRIMEIRO_NOME'].apply(get_sex, df=df)

Write File

In [19]:
df_data.to_csv('data/process/data_sexo.csv', encoding='latin1')

In [20]:
df_data = pd.read_csv('data/process/data_sexo.csv', index_col='Unnamed: 0')
df_data = df_data[df_data['sexo'] != 0]

In [23]:
print len(df_data), len(pd.read_csv('data/process/primeiro_nome.csv', index_col='Unnamed: 0'))
df_data.head()

 557914 590670


,NOME,ID_SERVIDOR_PORTAL,REMUNERACAO_BASICA_BRUTA,PRIMEIRO_NOME,sexo
0,PAULO MARTINS SOARES,1000000,"6779,03",paulo,0.999404
1,ANALIA BATISTA,1000001,"9151,22",analia,-1.000000
2,ARY DE NOROES FILHO,1000002,"6012,61",ary,1.000000
3,MOISES ALBERTO CALLE AGUIRRE,1000004,"11554,56",moises,0.996205
4,LAURA JANAINA DIAS AMATO,1000005,"10769,82",laura,-1.000000


Get names that have high probability of being of that sex: abs(sexo) > 0.6

In [24]:
df_data = df_data[abs(df_data['sexo']) > 0.6]
len(df_data)

553672

Transform remuneracao in number

In [25]:
def to_number(x):
    try:
        return float(x.replace(",","."))
    except:
        print x
df_data['REMUNERACAO_BASICA_BRUTA'] = df_data['REMUNERACAO_BASICA_BRUTA'].apply(to_number)

Put good names on sexo numbers

In [26]:
def namesexo(x):

    if x > 0.6:
        return 'Masc'
    elif x < 0.6:
        return 'Fem'
    
df_data['sexo2'] = df_data['sexo'].apply(namesexo)

In [28]:
df_data.to_csv('data/final/data_sexo_final.csv', encoding='latin1')